# Apresentação:

O objetivo desse código é estudar sobre a preparação de textos com o [keras](https://keras.io/api/), uma API de aprendizado profundo escrita em Python e capaz de ser executada em [JAX](https://jax.readthedocs.io/en/latest/quickstart.html), [TensorFlow](https://www.tensorflow.org/?hl=pt-br) ou [PyTorch](https://pytorch.org/docs/stable/index.html), dado que, como já visto, não se alimenta modelos com *strings*, afinal, operações matemáticas é uma propriedade dos números. Dito isso, a ideia aqui é ver sobre:

* Os métodos convenientes que pode-se usar para preparar rapidamente os dados de texto.
* A API Tokenizer que pode ser ajustada nos dados de treinamento e usada para codificar documentos de treinamento, validação e teste.
* A variedade de 4 diferentes esquemas de codificação de documentos oferecidos pela API Tokenizer.

In [16]:
import keras
import pandas as pd
print(keras.__version__)

2.10.0


#  Splicando palavras com `text_to_word_sequence`

Aqui continuamos com a ideia de tokenizar o texto em palavras, é uma forma arcaica de resolver essa questão, mas é útil no processo de estudo, por ser fácil de entender o processo. **Keras** fornece a função `text_to_word_sequence()` que você pode usar para dividir o texto em uma lista de palavras. Por padrão, esta função faz automaticamente três coisas:

* Divide palavras por espaço;
* Filtra pontuação;
* Converte texto em *lowercase* `(lower=True)`

A ideia aqui é que todo o processo de tratamento de *string* feita anteriormente é facilitada aqui. Deste modo, essa função do Keras facilita muito o trabalho. Dito isso, é importante dizer que todos os parâmetros apontados podem ser alterados para que função não faça tais mudanças.

In [2]:
# Importando função:
from keras.preprocessing.text import text_to_word_sequence

In [32]:
# Frase:
text = 'The quick brown fox jumped over the lazy dog.'

# Tokenização da frase:
result = text_to_word_sequence(text)
print(result, len(result))

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog'] 9


#  Codificando com `one-hot`

Como já dito, esse tipo de codificação é um método antigo de codificação, que gera matrizes muito esparças, todavia, a título de estudo codificar dessa forma. O Keras fornece a função `one_hot()`, que é diferente de bibliotecas como pandas, dado que aqui, não é simplemente zero ou um a saída, mas sim um **vetor números inteiro**. A função é um invólucro para a função `hashing_trick()`. A função retorna uma versão codificada em inteiros do documento. O uso de uma função de hash significa que **pode haver colisões** e **nem todas as palavras serão atribuídas a valores inteiros únicos**.

Assim como a função `text_to_word_sequence()`, a função `one_hot()` converterá o texto para minúsculas, filtrará a pontuação e dividirá as palavras com base em espaços em branco.

**Importante**

Essa função foi descontinuada, de modo que a própria biblioteca não recomenda mais a sua utilização. Para mais informações, `help(one_hot)`.

In [7]:
# Estimando o tamanho do vocabulário:
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(words,vocab_size)

{'lazy', 'jumped', 'the', 'brown', 'quick', 'fox', 'dog', 'over'} 8


In [10]:
# Importando a função one-hot
from keras.preprocessing.text import one_hot

In [30]:
# Codificação com inteiros:
resultEnconde = one_hot(text, round(vocab_size*1.3))
print(resultEnconde,len(resultEnconde))

[3, 8, 5, 2, 3, 9, 3, 4, 9] 9


In [35]:
# Visualizando Resposta:
df = pd.concat([pd.DataFrame(result, columns=["Palavras"]),
                pd.DataFrame(resultEnconde,columns=["Codificação One_Hot"])],axis=1) # Note que de fato há colisões

df

,Palavras,Codificação One_Hot
0,the,3
1,quick,8
2,brown,5
3,fox,2
4,jumped,3
5,over,9
6,the,3
7,lazy,4
8,dog,9


# Hash Encoding with `hashing_trick`

Uma limitação das codificações baseadas em inteiros e contagens é que elas precisam manter um vocabulário de palavras e seu mapeamento para inteiros. Uma alternativa a essa abordagem é usar uma função de **hash unidirecional** para converter palavras em inteiros. Isso evita a necessidade de acompanhar um vocabulário, o que é mais rápido e requer menos memória.

O Keras fornece a funão `hashing_trik()` que tokeniza e, em seguida, codifica o documento em inteiros, assim como a função `one_hot()`. Ela oferece mais flexibilidade, permitindo que você especifique a função de hash como hash (o padrão) ou outras funções de hash, como a função `md5` embutida ou sua própria função.

In [25]:
# Imporando a função de hash:
from keras.preprocessing.text import hashing_trick

In [26]:
# estimate the size of the vocabulary
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(words,vocab_size)

{'lazy', 'jumped', 'the', 'brown', 'quick', 'fox', 'dog', 'over'} 8


In [29]:
# integer encode the document
resultHashCodification = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
print(result,len(resultHashCodification)) # Note que continua havendo colisão

[6, 4, 1, 2, 7, 5, 6, 2, 6] 9


In [36]:
# Visualizando Resposta:
df = pd.concat([df, pd.DataFrame(resultHashCodification,columns=["Codificação Hash"])],axis=1) # Note que de fato há colisões

df

,Palavras,Codificação One_Hot,Codificação Hash
0,the,3,6
1,quick,8,4
2,brown,5,1
3,fox,2,2
4,jumped,3,7
5,over,9,5
6,the,3,6
7,lazy,4,2
8,dog,9,6


In [43]:
# Avaliando estatísticas descritivas:
df.describe().loc["mean":"std"]

,Codificação One_Hot,Codificação Hash
mean,5.111111,4.333333
std,2.803767,2.179449


# Tokenizer API:

O Keras fornece uma API mais sofisticada para preparar texto que pode ser ajustada e reutilizada para preparar vários documentos de texto. Esta pode ser a abordagem preferida para grandes projetos. O Keras fornece a classe `Tokenizer` para preparar documentos de texto para deep learning. O Tokenizer deve ser construído e, em seguida, ajustado em documentos de texto bruto ou documentos de texto codificados em inteiros.

In [44]:
# Importando Classe Tokenizer:
from keras.preprocessing.text import Tokenizer

In [45]:
# Definindo 5 frases:
docs = ['Well done!', 'Good work', 'Great effort', 'nice work', 'Excellent!']

# Instanciando Tokenizer:
t = Tokenizer()

# Ajustando o tokenizer aos documentos
t.fit_on_texts(docs)

Uma vez ajustado, o `Tokenizer` fornece 4 atributos que podem ser usados para consultar o que foi
aprendido sobre os documentos:

* **contagem de palavras:** Um dicionário de palavras e suas contagens.
* **word docs:** Uma contagem inteira do número total de documentos que foram usados ​​para ajustar o
Tokenizador.
* **índice de palavras:** Um dicionário de palavras e seus números inteiros atribuídos exclusivamente.
* **contagem de documentos:** Um dicionário de palavras e em quantos documentos cada uma apareceu.

In [49]:
# Sumário do que foi aprendido:
print(t.word_counts, "\n") # Nesse caso: Bigram, ou digram
print(t.document_count, "\n")
print(t.word_index, "\n")
print(t.word_docs, "\n")

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)]) 

5 

{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8} 

defaultdict(<class 'int'>, {'done': 1, 'well': 1, 'good': 1, 'work': 2, 'effort': 1, 'great': 1, 'nice': 1, 'excellent': 1}) 



Uma vez que o `Tokenizer` foi ajustado nos dados de treinamento, ele pode ser usado para codificar documentos nos conjuntos de dados de treinamento ou teste. O método `texts_to_matrix()` do `Tokenizer` pode ser usado para criar um vetor por documento fornecido como entrada. O comprimento dos vetores é o tamanho total do vocabulário. Esta função fornece um conjunto de esquemas de codificação de texto do modelo **bag-of-words** que podem ser fornecidos através de um argumento de modo para a função. Os modos disponíveis incluem:

* **binary:** Indica se cada palavra está presente no documento. Este é o padrão.
* **count:** A contagem de cada palavra no documento.
* **tfidf:** A pontuação de Frequência de Termo-Inversa Frequência de Documento **(TF-IDF)** para cada palavra no documento.

In [51]:
# texto com codificação inteira
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

[[0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


Esse é o nosso "*One-hot-encoding*" tradicional, deste modo.